# Chapter09 LangGraph

In [1]:
from dotenv import load_dotenv

loaded = load_dotenv()


## 9.1 Execute a simple graph

### Role Definition

In [2]:
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確でわかりやすい回答を提供してください。"
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。"
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。"
    }
}

### State Definition

Once the types for the graph’s states have been defined, that definition is passed to the StateGraph class, which instantiates the graph.

- **Base type**: `list[str]` — the `messages` field is essentially a list of strings.

- **Metadata `operator.add`**: tells LangGraph / Pydantic that, when two `State` objects are merged, the two lists should be concatenated with `+` instead of one list replacing the other.


In [3]:
import operator
from typing import Annotated

from pydantic import BaseModel, Field

class State(BaseModel):
    query: str = Field(
        ..., description="ユーザーからの質問"
    )
    current_role: str = Field(
        default="", description="選定された回答ロール"
    )
    # `Annotated[] allows users to add annotation -- `operator.add`.
    # `operator.add` tells LangGraph that, when two `State` objects are merged, the two lists should be concatenated with `+` instead of one list replacing the other.
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="回答履歴" 
    )
    current_judge: bool = Field(
        default=False, description="品質チェックの結果"
    )
    judgement_reason: str = Field(
        default="", description="品質チェックの判定理由"
    )

### Instantiate a Chat Model

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm = llm.configurable_fields(max_tokens=ConfigurableField(id="max_tokens"))

### `Selection Node`

In [5]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]: # Return a dict whose keys are str and whose values are not decided
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
        """質問を分析し、最も適切な回答担当ロールを選択してください。

        選択肢：
        {role_options}

        回答は選択肢の番号（１、２、または３）のみを返してください。
        
        質問：{query}
        """.strip()
    )
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})
    
    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

### `Answering Node`

In [6]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
        """あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。
        
        役割の詳細：
        {role_details}
        
        質問：{query}
        
        回答：""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

### `Check Node`

In [7]:
from langchain.output_parsers import PydanticOutputParser

class Judgement(BaseModel):
    reason: str = Field(default="", description="判定理由")
    judge: bool = Field(default=False, description="判定結果")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
        """以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。また、その判定理由も説明してください。
        
        ユーザーからの質問：{query}
        回答：{answer}
        """.strip()
    )


    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

### Instantiate a graph

The StateGraph class is used in Lang Graph to define the graph’s structure and manages the nodes and edges that compose the workflow.

In [8]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

### Add nodes

In [9]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

### Set the entry node

In [10]:
workflow.set_entry_point("selection")

### Define edges

In [11]:
# Connect `selection` node to `answering` node
workflow.add_edge("selection", "answering")
# Connect `asnwering` node to `check` node
workflow.add_edge("answering", "check")

### Define conditional edges

In [12]:
from langgraph.graph import END

workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

### Compile the graph

In [13]:
compiled = workflow.compile()

In [14]:
initial_state = State(query="生成AIについて教えてください")
result = compiled.invoke(initial_state)

In [15]:
print(result["messages"][-1])

生成AI製品エキスパートとしてお答えします。

生成AI（生成的人工知能）は、人工知能の一分野であり、テキスト、画像、音声、動画などの新しいコンテンツを生成する能力を持つモデルを指します。これらのモデルは、大量のデータを学習し、そのパターンを理解することで、新しいデータを生成することができます。

代表的な生成AIの技術には、以下のようなものがあります：

1. **GPT（Generative Pre-trained Transformer）**: テキスト生成に特化したモデルで、自然言語処理の分野で広く利用されています。GPT-3やGPT-4などのバージョンがあり、文章の生成、翻訳、要約などに活用されています。

2. **GAN（Generative Adversarial Networks）**: 画像生成において特に有名な技術で、2つのニューラルネットワーク（生成者と識別者）が競い合うことで、非常にリアルな画像を生成することができます。

3. **VAE（Variational Autoencoders）**: データの潜在的な特徴を学習し、新しいデータを生成するために使用されるモデルです。主に画像や音声の生成に利用されます。

生成AIは、クリエイティブなコンテンツの制作、デザインの自動化、カスタマーサービスのチャットボット、教育コンテンツの生成など、さまざまな分野で応用されています。しかし、倫理的な問題やデータの偏り、プライバシーの懸念などもあり、これらの課題に対処することが重要です。

生成AIの進化は急速であり、今後も新しい技術や応用が登場することが期待されています。


### Visualize the graph

In [16]:
from IPython.display import Image

Image(compiled.get_graph().draw_png())

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

## 9.2 Checkpoint

- **super-step** – the moment after a node has finished executing and the framework has determined which node will run next.

- When a *checkpointer* is supplied at compile time, the framework automatically invokes it at every super-step to save a `StateSnapshot`.

- Consequently, a checkpoint is always written whenever control moves from one node to the next, in addition to an initial snapshot before the graph starts.


In [ ]:
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class State(BaseModel):
    query: str
    messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

def add_message(state: State) -> dict[str, Any]:
    additional_messages = []
    if not state.messages:
        additional_messages.append(
            SystemMessage(content="あなたは最小限の応答をする対話エージェントです。")
        )
    additional_messages.append(HumanMessage(content=state.query))
    return {"messages": additional_messages}

def llm_response(state: State) -> dict[str, Any]:
    llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
    ai_message = llm.invoke(state.messages)
    return {"messages": [ai_message]}

In [ ]:
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
    """
    This function is used to get the CheckpointTuple and print the information of Checkpoint and metadata.
    """
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("チェックポイントデータ：")
    pprint(checkpoint_tuple.checkpoint)
    print("\nメタデータ：")
    pprint(checkpoint_tuple.metadata)

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)

graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

# Instantiate a checkpointer
checkpointer = MemorySaver()

compiled_graph = graph.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "example-1"}}
user_query = State(query="私の好きなものはずんだ餅です。覚えておいてね。")
first_response = compiled_graph.invoke(user_query, config)
print(first_response)

{'query': '私の好きなものはずんだ餅です。覚えておいてね。', 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}), AIMessage(content='了解です。ずんだ餅が好きなんですね。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 48, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'stop', 'logprobs': None}, id='run--29aa34cc-f491-4ada-a2a2-246a058e2091-0', usage_metadata={'input_tokens': 48, 'output_tokens': 13, 'total_tokens': 61})]}


In [ ]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f072c39-4235-67b4-8002-ff2daa3d3ee0'}}, checkpoint={'v': 4, 'ts': '2025-08-06T12:47:57.801857+00:00', 'id': '1f072c39-4235-67b4-8002-ff2daa3d3ee0', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.16516703335109273', 'query': '00000000000000000000000000000002.0.16516703335109273', 'messages': '00000000000000000000000000000004.0.6129457775643027', 'branch:to:add_message': '00000000000000000000000000000003.0.3710934076192379', 'branch:to:llm_response': '00000000000000000000000000000004.0.6129457775643027'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.8306737082457146'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000002.0.16516703335109273'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000003.0.3710934076192379'}}, 'channel_values': {'query': '私の好きなものはずんだ餅です。覚

In [ ]:
print_checkpoint_dump(checkpointer, config)

チェックポイントデータ：
{'channel_values': {'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='了解です。ずんだ餅が好きなんですね。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 48, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'stop', 'logprobs': None}, id='run--29aa34cc-f491-4ada-a2a2-246a058e2091-0', usage_metadata={'input_tokens': 48, 'output_tokens': 13, 'total_tokens': 61})],
                    'query': '私の好きなものはずんだ餅です。覚えておいてね。'},
 'channel_v

### Second response

In [ ]:
user_query = State(query="私の好物は何か覚えてる？")
second_response = compiled_graph.invoke(user_query, config)
print(second_response)

{'query': '私の好物は何か覚えてる？', 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}), AIMessage(content='了解です。ずんだ餅が好きなんですね。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 48, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'stop', 'logprobs': None}, id='run--29aa34cc-f491-4ada-a2a2-246a058e2091-0', usage_metadata={'input_tokens': 48, 'output_tokens': 13, 'total_tokens': 61}), HumanMessage(content='私の好物は何か覚えてる？', additional_kwargs={}, response_metadata={}), AIMessage(content='ずんだ餅です。', additional_kwargs={'refusal': N

In [ ]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f072c56-3998-6fee-8006-b12a2c39d3e1'}}, checkpoint={'v': 4, 'ts': '2025-08-06T13:00:55.361731+00:00', 'id': '1f072c56-3998-6fee-8006-b12a2c39d3e1', 'channel_versions': {'__start__': '00000000000000000000000000000006.0.24925599108332874', 'query': '00000000000000000000000000000006.0.24925599108332874', 'messages': '00000000000000000000000000000008.0.4414309472358884', 'branch:to:add_message': '00000000000000000000000000000007.0.9108803549024735', 'branch:to:llm_response': '00000000000000000000000000000008.0.4414309472358884'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000005.0.2501419612637731'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000006.0.24925599108332874'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000007.0.9108803549024735'}}, 'channel_values': {'query': '私の好物は何か覚えてる？', '

In [ ]:
print_checkpoint_dump(checkpointer, config)

チェックポイントデータ：
{'channel_values': {'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='私の好きなものはずんだ餅です。覚えておいてね。', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='了解です。ずんだ餅が好きなんですね。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 48, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'stop', 'logprobs': None}, id='run--29aa34cc-f491-4ada-a2a2-246a058e2091-0', usage_metadata={'input_tokens': 48, 'output_tokens': 13, 'total_tokens': 61}),
                                 HumanMessage(content='私の好物は何か覚えてる？',

### Try other thread_id

In [ ]:
config = {"configurable": {"thread_id": "example-2"}}
user_query = State(query="私の好物は何？")
other_thread_response = compiled_graph.invoke(user_query, config)
print(other_thread_response)

{'query': '私の好物は何？', 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の好物は何？', additional_kwargs={}, response_metadata={}), AIMessage(content='わかりません。あなたの好物は何ですか？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 36, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ff25b2783a', 'finish_reason': 'stop', 'logprobs': None}, id='run--1b407575-e65c-42c1-88cf-8064dc5a6226-0', usage_metadata={'input_tokens': 36, 'output_tokens': 14, 'total_tokens': 50})]}


In [ ]:
print_checkpoint_dump(checkpointer, config)

チェックポイントデータ：
{'channel_values': {'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='私の好物は何？', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='わかりません。あなたの好物は何ですか？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 36, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ff25b2783a', 'finish_reason': 'stop', 'logprobs': None}, id='run--1b407575-e65c-42c1-88cf-8064dc5a6226-0', usage_metadata={'input_tokens': 36, 'output_tokens': 14, 'total_tokens': 50})],
                    'query': '私の好物は何？'},
 'channel_versions': {'__start__': '000000